In [1]:
import requests
import json

import re
import networkx as nx
import tqdm
import collections

import spacy
nlp = spacy.load('de') #load spacy model

# for WordClouds 
import matplotlib.pyplot as plt
from wordcloud import WordCloud

with open("../../bundestagsreden parser/speeches_20.jsonl",'r',encoding = "utf8") as fp:
    data = list(fp)
speeches = []
for line in data:
    speeches.append(json.loads(line))



In [48]:
def filter_for(what, search_terms, speeches):
    filtered_speeches = []
    if what == 'text':
        search_terms_low = []
        for st in search_terms:
            search_terms_low.append(st.lower())
        for speech in speeches:
            match = [st in speech[what].lower() for st in search_terms_low]
            #if all(st == True for st in match):
            if any(st in speech[what] for st in search_terms):
                #print(match)
            #if ( search_terms in speech[what] ):
                filtered_speeches.append(speech)
    else:
        for speech in speeches:
            if ( speech[what].lower() in set(search_terms_low) ):
                filtered_speeches.append(speech)
        
    filtered_speeches.sort(key = lambda x:x['date'])   
    return filtered_speeches


#focal_terms = ['Digitalisierung','Zusammenhalt','Demokratie']
#focal_terms = ['Digitalisierung']
#focal_terms = ['Plattform','Demokratie']
#focal_terms = ['extrem','Plattform']
#focal_terms = ['plattform','demokratie']
#focal_terms = ['Plattform','Meinung']
#focal_terms = ['Netzwerk','Meinung']
focal_terms = ['Ukraine']
subset = filter_for('text', focal_terms, speeches)
len(subset)

770

In [49]:
sentences = []
#focal_terms_sen = ['Netzwerk','sozial']
focal_terms_sen = ['Russland']
window = 10
for rede in tqdm.tqdm(subset):
    doc = nlp(rede["text"])
    #print(rede['name'])
    for sent in doc.sents:
        #print(sent.text)
        if all(ft.lower() in sent.text.lower() for ft in focal_terms_sen):
        #if focal_terms[0].lower() in sent.text.lower() and focal_terms[1].lower() in sent.text.lower():
        #for focus in focal_terms:    
            #if focus in sent.text:
            sentences.append(sent)
        
        
len(sentences)

100%|██████████| 770/770 [01:08<00:00, 11.20it/s]


844

In [4]:
sentences[:2]

[Zum einen – und das ist die Mehrheitsmeinung im deutschen Volke – geht es um die Lesbarkeit von Drucksachen durch Verzicht auf die sogenannte Gendersprache.,
 Also, Sie können was für unser Volk, für die Mehrheit des Volkes, tun.]

In [50]:
relevantPOS = ['NOUN','ADJ','PROPN']
sentencesNN = []
words = []
for sen in sentences:
    lem = []
    for token in sen:
        if token.pos_ in relevantPOS:
            lem.append(token.lemma_)
    sentencesNN.append(lem)
    words.extend(lem)
#sentencesNN
len(words)


6378

In [55]:
zähler = collections.Counter(words).most_common()
#print(zähler)

words_clean = []
#print(focal_terms_sen)
for word in words:
    if focal_terms_sen[0] in word or focal_terms[0] in word or 'Herr' in word or 'Dame' in word or 'Kollege' in word or 'Kollegin' in word or "/" in word:
        pass
    else:
        words_clean.append(word)
    
#print(len(words_clean))
zähler = collections.Counter(words_clean).most_common()
#print(zähler)

num_words = 200
count = 0
new_words = []
for word in zähler:
    #print(word[0])
    if count < num_words:
        new_words.append(word[0])
    count = count + 1    
        
new_words        

['Krieg',
 'Europa',
 'Land',
 'Angriffskrieg',
 'Deutschland',
 'Putin',
 'Sanktion',
 'Mensch',
 'ander',
 'Jahr',
 'Prozent',
 'groß',
 'russisch',
 'militärisch',
 'NATO',
 'Staat',
 'Tag',
 'letzt',
 'deutsch',
 'Sicherheit',
 'europäisch',
 'China',
 'Abhängigkeit',
 'Putins',
 'klaren',
 'wichtig',
 'Welt',
 'unabhängig',
 'Seite',
 'USA',
 'Weg',
 'gemeinsam',
 'Angriff',
 'wirtschaftlich',
 'Teil',
 'Gas',
 'Grenze',
 'europäische',
 'Regierung',
 'Situation',
 'Aggression',
 'abhängig',
 'massiv',
 'gut',
 'Westen',
 'Druck',
 'Präsident',
 'Beispiel',
 'völkerrechtswidrigen',
 'weit',
 'aktuell',
 'fossil',
 'Energie',
 'Konflikt',
 'Demokratie',
 'Friede',
 'Frage',
 'Bundesregierung',
 'Blick',
 'Friedensordnung',
 'russische',
 'Konsequenz',
 'Ende',
 'Öl',
 'Überfall',
 'Krim',
 'Union',
 'deutlich',
 'Herausforderung',
 'Zeit',
 'Woche',
 'völkerrechtswidrige',
 'Folge',
 'stark',
 'Interesse',
 'Volk',
 'Bundeskanzler',
 'Bundestag',
 'Belarus',
 'Zukunft',
 'Bedrohung

In [56]:
nodes=[]
curid=1
for word in set(new_words):
    node = {
    'id' :  curid,
    'name' : word
    }
    nodes.append(node)
    curid=curid+1
    
graph = {
    'directed': False,
    'graph': 'word_graph',
    'links': [],
    'nodes': nodes
}

links = []
linkedwords =[]
linkedids =[]
lx = 0;
for wx1,w1 in enumerate(nodes):
    #print(wx1)
    for wx2,w2 in enumerate(nodes):
        if(w2['id'] > w1['id']):
            for sen in sentencesNN:
                if w1['name'] in sen and w2['name'] in sen:
                    weight = len([ele for ele in linkedwords if ele == (' '.join([w1['name'],w2['name']]))])
                    #    links[]
                    #else:
                    #print(weight)
                    linkedwords.append(' '.join([w1['name'],w2['name']]))
                    #linkedids[ w1['id'] ] , w2['id'] )
                    link_dict = {
                    'source':w1['id'],
                    'target':w2['id'],
                    'sourceWD':w1['name'],
                    'targetWD':w2['name'],
                    'weight': weight+1      
                    }
                    #print(link_dict)
                    links.append(link_dict)
                    graph['links'].append(link_dict)
               
                    #for link in links:

#linkedwords
zähler = collections.Counter(linkedwords).most_common()
print(len(linkedwords),len(links))
#graph['links']=links
#print(zähler)

3792 3792


In [57]:
print(len(graph['links']),
len(graph['nodes']))
#for link in graph['links']:
#    if link['weight']>1:
        #print(link)

3792 200


In [58]:


graphforgephi = nx.Graph()
for node in tqdm.tqdm(graph['nodes']):
    graphforgephi.add_node(node['id'],name = node['name']);
print('nodes done')
for link in tqdm.tqdm(graph['links']):
    #weight = all((' '.join([w1['name'],w2['name']]) in linkedwords)
    #print(weight)         
    graphforgephi.add_edge(link['source'],link['target'],weight=link['weight'])
print('links done')   
nx.write_gexf(graphforgephi, "graphforgephi.gexf")
print('save done')

100%|██████████| 3792/3792 [00:00<00:00, 408255.06it/s]

nodes done
links done
save done
